In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
def comparison_plot(x, y):
    plt.plot(x, x, 'r')
    plt.plot(x, y, 'k.')
    
#comparison_plot(np.random.rand(10), np.random.rand(10))

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

https://www.tensorflow.org/guide/eager

In [5]:
tf.executing_eagerly()

True

In [6]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

hello, [[4.]]


In [7]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [8]:
b = tf.add(a, 1)
print(b)
print(type(a))
print(type(b))
print(type(m))

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [9]:
c = np.multiply(a, a)
print(c)
print(type(c))

c = np.matmul(a, a)
print(c)
print(type(c))

c = a * a
print(c)
print(type(c))

c = a @ a
print(c)
print(type(c))

c = (a @ a).numpy()
print(c)
print(type(c))

[[ 1  4]
 [ 9 16]]
<class 'numpy.ndarray'>
[[ 7 10]
 [15 22]]
<class 'numpy.ndarray'>
tf.Tensor(
[[ 1  4]
 [ 9 16]], shape=(2, 2), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[ 7 10]
 [15 22]], shape=(2, 2), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
[[ 7 10]
 [15 22]]
<class 'numpy.ndarray'>


In [10]:
def get_loss(w):
    return w * w

w = tf.Variable([[1.0, 2.0], [1.0, 2.0]])
with tf.GradientTape() as tape:
    loss = get_loss(w)
    loss_double = 2 * loss

grad = tape.gradient(loss, w)
print(grad)

# Error, since you can only call gradient once on non-persistent tapes
if False:
    grad_double = tape.gradient(loss_double, w)
    print(grad_double)

tf.Tensor(
[[2. 4.]
 [2. 4.]], shape=(2, 2), dtype=float32)


In [11]:
print('\nWithout watching constant x:')
x = tf.constant(2.0)
with tf.GradientTape() as tape:
    z = x**2
print(tape.gradient(z, x))

print('\nWatching constant x:')
x = tf.constant(2.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    z = x**2
print(tape.gradient(z, x))


print('\nx is a variable:')
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
    z = x**2
print(tape.gradient(z, x))


Without watching constant x:
None

Watching constant x:
tf.Tensor(4.0, shape=(), dtype=float32)

x is a variable:
tf.Tensor(4.0, shape=(), dtype=float32)


In [12]:
# Names of constants seem to be discarded in eager execution
print(tf.Variable(3.0, name='Broom'))
print(tf.constant(3.0, name='Broom'))

<tf.Variable 'Broom:0' shape=() dtype=float32, numpy=3.0>
tf.Tensor(3.0, shape=(), dtype=float32)


In [13]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
    tape.watch(x)
    y = x**2
    z = y**2 + x

# With directed graphs you can compute partial derivatives with respect
# to internal variables.
# y = 3 * 3, and dz / dy = 2 * y
print(tape.gradient(z, y))

with tf.GradientTape() as tape:
    tape.watch(x)
    y = x**2
    z = y**2 + x

# 4 * x^3 + 1 = 4 * 27 + 1 
print(tape.gradient(z, x))
print(4 * 27 + 1 )

tf.Tensor(18.0, shape=(), dtype=float32)
tf.Tensor(109.0, shape=(), dtype=float32)
109


In [14]:
# You can assign variables but not constants
x = tf.Variable([2.0, 3.0], name='groundhog')
print(x)
x.assign([3.0, 4.0])
print(x)

# Doesn't work
x = tf.constant([2.0, 3.0])
try:
    x.assign([3.0, 4.0])
except Exception as inst:
    print(inst)

<tf.Variable 'groundhog:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>
<tf.Variable 'groundhog:0' shape=(2,) dtype=float32, numpy=array([3., 4.], dtype=float32)>
'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'


In [15]:
# Be careful with assignment and constants!
x = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x**2
    print('inside 1:', x)
    
    # This appears to silently create a new variable, and it is with respect
    # to this variable that you get a derivative.
    x = y
    print('inside 2:', x)
    z = x**2

# You might expect 4 * 3^3 = 4 * 27, but you get 2 * 9 = 18.
print(tape.gradient(z, x))

inside 1: tf.Tensor(3.0, shape=(), dtype=float32)
inside 2: tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)


In [16]:
# Be careful with assignment and variables!
x = tf.Variable(3.0, name='woodchuck')
with tf.GradientTape() as tape:
    y = x**2
    print('inside 1, x:', x)
    print('inside 1, y:', y)
    
    # This appears to silently create a new variable, and it is with respect
    # to this variable that you get a derivative.
    x = y
    print('inside 2, x:', x)
    z = x**2

# You might expect 4 * 3^3 = 4 * 27, but you get 2 * 9 = 18.
print(tape.gradient(z, x))

inside 1, x: <tf.Variable 'woodchuck:0' shape=() dtype=float32, numpy=3.0>
inside 1, y: tf.Tensor(9.0, shape=(), dtype=float32)
inside 2, x: tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)


In [17]:
# Be careful with assignment and variables!
x = tf.Variable(3.0, name='woodchuck')
with tf.GradientTape() as tape:
    y = x**2
    x.assign(y)
    z = x**2

# You might expect 4 * 3^3 = 4 * 27 = 108, but you get 2 * 9 = 18.
print(tape.gradient(z, x))
# 18

tf.Tensor(18.0, shape=(), dtype=float32)


In [18]:
# Contrast with autograd.

import autograd
from autograd import numpy as anp

def loopy_fun(x):
    y = anp.power(x, 2)
    x = y
    z = anp.power(x, 2)
    return z

print(autograd.grad(loopy_fun)(3.0))
# 108

108.0


In [19]:
# Be careful with assignment and variables!
@tf.function
def tf_loopy_fun(x):
    y = x**2
    tf.print(x)
    x.assign(y)
    tf.print(x)
    z = x**2
    return z

x = tf.Variable(3.0, name='woodchuck')
with tf.GradientTape() as tape:
    z = tf_loopy_fun(x)
print(tape.gradient(z, x).numpy())
# 18 again

3
9
18.0


In [20]:
# Here you get errors

def get_tf_grad(fun, x):
    x = tf.Variable(3.0, name='woodchuck')
    with tf.GradientTape() as tape:
        z = fun(x)
    try:
        grad = tape.gradient(z, x).numpy()
    except Exception as err:
        print(err)
        grad = None
    return grad

def tf_fun1(x):
    y = x
    return y ** 2

def tf_fun2(x):
    y = tf.Variable(10.0)
    y.assign(x)
    return y ** 2

def tf_fun3(x):
    y_outer.assign(x)
    return y ** 2

def tf_fun4(x):
    y = tf.Variable(10.0)
    y.assign_add(x)
    return y ** 2

def tf_fun5(x):
    y_outer.assign_add(x)
    return y ** 2

y_outer = tf.Variable(10.0)
print("tf_fun1", get_tf_grad(tf_fun1, x))  # Works
print("tf_fun2", get_tf_grad(tf_fun2, x))  # Error
print("tf_fun3", get_tf_grad(tf_fun3, x))  # Error
print("tf_fun4", get_tf_grad(tf_fun4, x))  # Error
print("tf_fun5", get_tf_grad(tf_fun5, x))  # Error

tf_fun1 6.0
'NoneType' object has no attribute 'numpy'
tf_fun2 None
'NoneType' object has no attribute 'numpy'
tf_fun3 None
'NoneType' object has no attribute 'numpy'
tf_fun4 None
'NoneType' object has no attribute 'numpy'
tf_fun5 None


In [21]:
# Declare persistent to get multiple derivatives
x = tf.constant(3.0, name='intial')
with tf.GradientTape(persistent = True) as tape:
    tape.watch(x)
    y = x**2
    z = y**2

print(tape.gradient(z, x))
print(tape.gradient(z, y))

del tape

tf.Tensor(108.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)


In [22]:
def get_mask(inds, x):
    output = tf.Variable(np.zeros_like(x.numpy()))
    for i in inds:
        output[i] = x[i]
    return output
     
x = tf.Variable(np.random.random((3, 4)))
inds = [(0, 1), (2, 3), (1, 1)]
try:
    get_mask(inds, x)
except Exception as err:
    print(err)

'ResourceVariable' object does not support item assignment


In [23]:
x = np.arange(12, dtype=np.float32).reshape((3, 4))
inds = [(0, 1), (2, 3), (1, 1)]

def get_mask(inds, x):
    x_sp = tf.sparse.SparseTensor(inds, [ x[i] for i in inds ], x.shape)
    x_sp = tf.sparse.reorder(x_sp)
    return x_sp

tf_x = tf.Variable(x)
x_sp = get_mask(inds, tf_x)

with tf.GradientTape() as tape:
    x_sp = get_mask(inds, tf_x)
    x_sp_sum = tf.sparse.reduce_sum(x_sp)
print(tape.gradient(x_sp_sum, tf_x))

tf.Tensor(
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]], shape=(3, 4), dtype=float32)


In [24]:
foo = tf.Variable(np.arange(12, dtype=np.float16).reshape((3, 4)))
# foo[1, 3] = 5 # Fails

# but this works:
print(foo)
print(foo[1, 3].assign(1000))
print(foo)

<tf.Variable 'Variable:0' shape=(3, 4) dtype=float16, numpy=
array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]], dtype=float16)>
<tf.Variable 'UnreadVariable' shape=(3, 4) dtype=float16, numpy=
array([[   0.,    1.,    2.,    3.],
       [   4.,    5.,    6., 1000.],
       [   8.,    9.,   10.,   11.]], dtype=float16)>
<tf.Variable 'Variable:0' shape=(3, 4) dtype=float16, numpy=
array([[   0.,    1.,    2.,    3.],
       [   4.,    5.,    6., 1000.],
       [   8.,    9.,   10.,   11.]], dtype=float16)>


In [25]:
# Doesn't work.
x = tf.Variable(np.arange(12, dtype=np.float32).reshape((3, 4)))
inds = [(0, 1), (2, 3), (1, 1)]

def get_mask2(inds, x):
    x_masked = tf.Variable(np.zeros(x.shape, dtype=np.float32))
    for i in inds:
        x_masked[i].assign(x[i])
    return x_masked

with tf.GradientTape() as tape:
    x_masked = get_mask2(inds, tf_x)
    x_mask_sum = tf.math.reduce_sum(x_masked)
print(tape.gradient(x_mask_sum, tf_x))

None


In [37]:
@tf.function
def f(x):
    y = x ** 2
    z = y ** 2
    return 0.5 * z

x = tf.Variable([2.0, 1.5])
g = f.get_concrete_function(x).graph
print(type(g))
g.get_operations()

<class 'tensorflow.python.framework.func_graph.FuncGraph'>


[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'ReadVariableOp' type=ReadVariableOp>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'pow_1/y' type=Const>,
 <tf.Operation 'pow_1' type=Pow>,
 <tf.Operation 'mul/x' type=Const>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'Identity' type=Identity>]